In [1]:
%%capture
!pip install sentencepiece
!pip install transformers

In [2]:
#DO ALL NECESSARY IMPORTS
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import CamembertConfig, CamembertModel, CamembertTokenizer, CamembertTokenizer, CamembertForSequenceClassification
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential               ##It is used to plot the frequency of the words which determines their size
from sklearn.preprocessing import MinMaxScaler



In [3]:
#use bert embedding but using camembert as model
model_name = 'camembert-base'
camembert_model = CamembertModel.from_pretrained(model_name)
tokenizer = CamembertTokenizer.from_pretrained(model_name, revision="main", sentencepiece_model="/usr/local/lib/python3.10/dist-packages")


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

def get_bert_embedding(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

def max_pool_embeddings(embeddings):
    return np.max(embeddings, axis=0)

def final_camembert(df):
    df['camembert_embedding'] =  df['sentence'].apply(lambda x: get_bert_embedding(x, camembert_model, tokenizer))
    df['cam_pooled_embedding'] = df['camembert_embedding'].apply(lambda x: max_pool_embeddings(x))
    return df

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Data-Kaggle

/content/drive/MyDrive/Data-Kaggle


In [7]:
import pandas as pd

test = pd.read_csv('https://github.com/eperroud/DataScienceProject/raw/b11a6530812e1311cfe11e2c7a6a20e1b544b796/data/final_test.csv')
training = pd.read_csv('https://github.com/eperroud/DataScienceProject/raw/b11a6530812e1311cfe11e2c7a6a20e1b544b796/data/final_training.csv')

In [ ]:
training.head(1)

,sentence,difficulty,encoded_diff,sentence_sp,tokens,tokens_no_stop,token_count_no_stop,token_count,nb_verbs,nb_nouns,nb_adj,nb_adv,words,tfidf_score_unigram,tfidf_score_bigram,cognate_count
0,Les coûts kilométriques réels peuvent diverger...,C1,4,Les coûts kilométriques réels peuvent diverger...,"['Les', 'coûts', 'kilométriques', 'réels', 'pe...","['coûts', 'kilométriques', 'réels', 'diverger'...",27,45,4,14,3,2,"['les', 'coûts', 'kilométriques', 'réels', 'pe...",27.043962,18.477813,1


In [8]:
training = final_camembert(training)

In [9]:
test = final_camembert(test)

In [10]:
pred_features = ['nb_nouns','nb_adj', 'token_count', 'token_count_no_stop', 'cognate_count' ]
#pred_features = []

In [11]:
X_cam = pd.DataFrame(training['cam_pooled_embedding'].tolist())
X = pd.concat([training[pred_features], X_cam], axis=1)
X.columns = X.columns.astype(str)
y = training[['encoded_diff']]
#scaller = MinMaxScaler()
#X = scaller.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50, shuffle=True)


In [12]:
X_train.shape

(3360, 773)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(X.shape[1], 128)
        self.linear2 = nn.Linear(128, 1)  # or more neurons if it's a classification task

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    def predict(self, X_to_predict):
        X_t = torch.tensor(X_to_predict.values, dtype=torch.float32)
        model.eval()
        with torch.no_grad():
            predictions = model(X_t)

        return predictions.numpy()


In [14]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.float32)

# Create a dataset and data loader
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [15]:
model = Net()

# Initialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function
criterion = nn.MSELoss()  # or another appropriate loss function

# Training loop
for epoch in range(15):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [16]:
predictions = model.predict(X_test)
y_predictions = np.minimum(np.round(predictions), 5).astype(int)
y_predictions = np.maximum(np.round(y_predictions), 0).astype(int)
y_predictions = pd.DataFrame(y_predictions, columns=['predictions'])
y_test = y_test.reset_index(drop=True)
# Do something with the predictions


In [22]:
accuracy_df = pd.concat([y_predictions, y_test], axis = 1, ignore_index= False )
accuracy_mask = accuracy_df['predictions'] == accuracy_df['encoded_diff']
accuracy = accuracy_mask.mean()
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
precision = precision_score(accuracy_df['encoded_diff'], accuracy_df['predictions'], average='weighted')
recall = recall_score(accuracy_df['encoded_diff'], accuracy_df['predictions'], average='weighted')
f1 = f1_score(accuracy_df['encoded_diff'], accuracy_df['predictions'], average='weighted')

print("Accuracy:", round(accuracy,2))
print("Precision:", round(precision,2))
print("Recall:", round(recall,2))
print("F1 Score:", round(f1,2))

Accuracy: 0.46
Precision: 0.51
Recall: 0.46
F1 Score: 0.45


In [ ]:
modelFull = Net()
X_cam = pd.DataFrame(training['cam_pooled_embedding'].tolist())
X_full = pd.concat([training[pred_features], X_cam], axis=1)
X_full.columns = X.columns.astype(str)
y_full = training[['encoded_diff']]
# Convert to PyTorch tensors
X_tensor_full = torch.tensor(X_full.values, dtype=torch.float32)
y_tensor_full = torch.tensor(y_full.values, dtype=torch.float32)

# Create a dataset and data loader
dataset_full = TensorDataset(X_tensor_full, y_tensor_full)
train_loader_full = DataLoader(dataset_full, batch_size=64, shuffle=True)

# Initialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function
criterion = nn.MSELoss()  # or another appropriate loss function

# Training loop
for epoch in range(15):
    for inputs, targets in train_loader_full:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

Retrain on full dataset

In [ ]:
X_cam = pd.DataFrame(test['cam_pooled_embedding'].tolist())
X_full = pd.concat([test[pred_features], X_cam], axis=1)
X_full.columns = X_full.columns.astype(str)
predictions_full = modelFull.predict(X_full)



In [ ]:
display(predictions_full)

array([[5.303759 ],
       [1.552356 ],
       [2.209244 ],
       ...,
       [5.1783233],
       [2.3266776],
       [2.5103903]], dtype=float32)

In [ ]:
test['difficulty encoded'] = np.minimum(np.round(predictions_full), 5).astype(int)
test['difficulty encoded'] = np.maximum(np.round(test['difficulty encoded']), 0).astype(int)

In [ ]:
test.head(2)

,id,sentence,sentence_sp,tokens,tokens_no_stop,token_count_no_stop,token_count,nb_verbs,nb_nouns,nb_adj,nb_adv,words,tfidf_score_unigram,tfidf_score_bigram,camembert_embedding,cam_pooled_embedding,difficulty encoded,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,Nous dûmes nous excuser des propos que nous eû...,"['Nous', 'dûmes', 'nous', 'excuser', 'des', 'p...","['dûmes', 'excuser', 'propos', 'eûmes', 'prono...",5,10,3,1,0,0,"['nous', 'dûmes', 'nous', 'excuser', 'des', 'p...",3.192917,2.331769,"[[0.047271907, 0.030453198, -0.022989253, 0.10...","[0.047271907, 0.030453198, -0.022989253, 0.109...",5,NaN
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,Vous ne pouvez pas savoir le plaisir que j'ai ...,"['Vous', 'ne', 'pouvez', 'pas', 'savoir', 'le'...","['pouvez', 'savoir', 'plaisir', 'recevoir', 'b...",7,16,4,1,2,2,"['vous', 'ne', 'pouvez', 'pas', 'savoir', 'le'...",12.660056,8.902162,"[[-0.0065091653, -0.11096406, -0.0380778, 0.06...","[-0.0065091653, -0.11096406, -0.0380778, 0.065...",2,NaN


In [ ]:
#function that cleans and return the final dataset to upload on kaggle
def prep_final_pred(df):
  number_to_level = {
    0: 'A1',
    1: 'A2',
    2: 'B1',
    3: 'B2',
    4: 'C1',
    5: 'C2'
}

  df['difficulty'] = df['difficulty encoded'].map(number_to_level)
  columns_to_keep = ['id', 'difficulty']
  final_df =  df[columns_to_keep]
  final_df.set_index('id', inplace=True)


  return final_df


In [ ]:
final_test = prep_final_pred(test)

In [ ]:
final_test.to_csv('submission.csv')

In [ ]:
final_test.head()

,difficulty
id,
0,C2
1,B1
2,B1
3,A2
4,C2


In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"


100% 8.30k/8.30k [00:00<00:00, 26.4kB/s]
Successfully submitted to Detecting the difficulty level of French texts